In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ======== 파라미터 ========
training_steps = 22000
display_step = 1000

In [3]:
# ======== 학습데이터 ========
xy = np.loadtxt('softmax_data.txt', unpack=True, dtype='float32')

# 학습시간, 해외거주(월)
x_data = xy[0:2]

# 영어 학점 : 0 : A, 1: B , 2: C, 3: D
y_data = xy[2:]

#  배열 위치 변환
X = x_data[:].T
print(X)
Y = y_data
print(Y.T)

[[54. 12.]
 [ 8.  0.]
 [30. 12.]
 [24. 15.]
 [46. 12.]
 [12.  0.]
 [20. 36.]
 [37. 12.]
 [40. 12.]
 [48. 24.]]
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]


In [4]:
# ======== W, B 초기값 설정 =========
#W = tf.Variable(tf.random.normal((2,1)))
#b = tf.Variable(np.random.randn(), name="bias")
W = tf.Variable(tf.zeros([2, 4]))
b = tf.Variable(1.0)
W

<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

In [5]:
X

array([[54., 12.],
       [ 8.,  0.],
       [30., 12.],
       [24., 15.],
       [46., 12.],
       [12.,  0.],
       [20., 36.],
       [37., 12.],
       [40., 12.],
       [48., 24.]], dtype=float32)

In [6]:
W

<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

In [7]:
tf.matmul(X, W) + b

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [8]:
hypothesis = tf.nn.softmax(tf.matmul(X, W)+b)

In [9]:
hypothesis

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]], dtype=float32)>

In [10]:
# ======== logistic Regression 에서 학습될 가설 ========
# Logistic regression (Wx + b).
def logistic_regression(x):
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(x, W) + b)

In [11]:
# ======== logistic Regression 에서 학습될 가설의 Cost Function ========
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

In [12]:
# ======== logistic Regression 에서 학습될 가설의 정확도 ========
# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).    
    correct_prediction = tf.equal(tf.argmax(y_pred), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
# ======== Gradient Descent Algorithm 에서 Step ========
learning_rate = 0.0006

In [14]:
# ======== 텐서플로우에 내장된 GradientDescentOptimizer ========
# Stochastic Gradient Descent Optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [15]:
# ======== GradientDescentOptimizer ========
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [16]:
# Run training for the given number of steps.
for step in range(1, training_steps + 1):
    # Run the optimization to update W and b values.
    run_optimization(X, Y.T)

    if step % display_step == 0:
        pred = logistic_regression(X)
        loss = cross_entropy(pred, Y.T)
        #acc = accuracy(pred, Y.T)
        #print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
        print("step: %i, loss: %f" % (step, loss))
        
        '''
        print("cost type: {}".format(type(cost)))
        print("W type: {}".format(type(W)))
        print("b type: {}".format(type(b)))
        print("pred type: {}".format(type(pred)))
        '''

step: 1000, loss: 4.712272
step: 2000, loss: 3.592767
step: 3000, loss: 3.168152
step: 4000, loss: 3.046665
step: 5000, loss: 2.964178
step: 6000, loss: 2.903743
step: 7000, loss: 2.857252
step: 8000, loss: 2.820227
step: 9000, loss: 2.789966
step: 10000, loss: 2.764725
step: 11000, loss: 2.743326
step: 12000, loss: 2.724942
step: 13000, loss: 2.708964
step: 14000, loss: 2.694944
step: 15000, loss: 2.682538
step: 16000, loss: 2.671481
step: 17000, loss: 2.661561
step: 18000, loss: 2.652612
step: 19000, loss: 2.644496
step: 20000, loss: 2.637100
step: 21000, loss: 2.630338
step: 22000, loss: 2.624123


In [17]:
# ======== 학습된 우리의 프로그램에 예측 문의 ========
sample = np.array([
    [30, 10] 
], dtype=np.float32)


# 30 시간 공부하고 10개월 해외에서 거주했을 경우 영어학점 예측
predictions = logistic_regression(sample)
print ("%i 시간, %i 개월 해외거주: %i " % (sample[0,0], sample[0,1], np.argmax(predictions.numpy())))

30 시간, 10 개월 해외거주: 2 
